In [ ]:
%matplotlib inline

import pandas as pd
import stumpy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib.patches import Rectangle
import plotly.graph_objects as go
import datetime as dt
import os
from glob import glob
import datetime
import sys
import subprocess
import pkg_resources
from helper_functions import *
# plt.style.use('https://raw.githubusercontent.com/TDAmeritrade/stumpy/main/docs/stumpy.mplstyle')

KITE CONNECT DB

DAILY TF

In [ ]:
start_ = '20221123'
end_ = '20221205'
Q_df = read_file_from_symbol('UCOBANK')

# Q_df = pd.DataFrame(filter_df_dates(Q_df, start_, end_)['pivot'])

In [ ]:
match_dict = dict()

for f in files:
    try:
        symbol_ = f.split('/')[-1].split('.')[0]
        print(symbol_)
        df = read_file_from_symbol(symbol_)
        df = n_month_gain(df, 1)
        df = n_month_gain(df, 3)
        df = n_month_gain(df, 6)
        df = adr(df)
        df['pivot'] = (df['high']+df['low']+df['close'])/3
        T_df = pd.DataFrame(df['pivot'])
        m = Q_df["pivot"].size
        # To set the exclusion zone to zero, set the denominator to np.`inf`
        stumpy.config.STUMPY_EXCL_ZONE_DENOM = np.inf

        top_N = stumpy.match(
            Q_df["pivot"],
            T_df["pivot"],
            max_distance=np.inf, # set the threshold to infinity include all subsequences
            max_matches=1,      # find the top 1 match
        )[0]

        stumpy.config.STUMPY_EXCL_ZONE_DENOM = 4 # Reset the denominator to its default value
        match_distance = top_N[0]
        match_idx = top_N[1]
        match_dict[symbol_] = [match_distance, match_idx, df.iloc[match_idx], df.iloc[match_idx+m]]
        print(symbol_, match_distance, match_idx)
        print('***')
    except:
        print('Error in', symbol_)

In [ ]:
match_dict_df = pd.DataFrame(match_dict).transpose()
match_dict_df.columns = ['distance','index', 'Other']
match_dict_df = match_dict_df.sort_values('distance')
# x = match_dict_df.iloc[0,-1]
match_dict_df
match_dict_df['Date'] = match_dict_df['Other'].apply(lambda x: x.Date)

Minute TF

In [ ]:
path = '/Users/yash/Desktop/Trading/code/raw'
os.chdir(path)

In [ ]:
start_ = '20221123'
end_ = '20221205'
field = 'close'

df = read_file_from_symbol3('AMBUJA', path)
# Q_df = pd.DataFrame(filter_df_dates(Q_df, start_, end_)[field])

In [ ]:
df_list = df.values.tolist()

In [ ]:
Q_df = pd.DataFrame(df[field])

In [ ]:
m = 60
mp = stumpy.stump(Q_df[field], m)
# mp = stumpy.gpu_stump(Q_df[field], m=m)  # Note that you'll need a properly configured NVIDIA GPU for this

In [ ]:
motif_idx = np.argsort(mp[:, 0])[0]

print(f"The motif is located at index {motif_idx}")

In [ ]:
df_motif = df.iloc[motif_idx:motif_idx+m,:]

fig = go.Figure(data=[go.Candlestick(open=df_motif['open'],
                high=df_motif['high'],
                low=df_motif['low'],
                close=df_motif['close'])])

fig.show()

In [ ]:
motifs = stumpy.motifs(Q_df[field],
    mp[:, 0],
    max_distance=1.0)

In [ ]:
for motif_idx in motifs[1][0]:
    print(df['Date'].iloc[motif_idx])
    df_motif = df.iloc[motif_idx:motif_idx+m,:]
    
    fig = go.Figure(data=[go.Candlestick(x=df_motif['Date'],
                    open=df_motif['open'],
                    high=df_motif['high'],
                    low=df_motif['low'],
                    close=df_motif['close'])])

    fig.show()

In [ ]:

fig = go.Figure(data=[go.Candlestick(x=df_motif['Date'],
                open=df_motif['open'],
                high=df_motif['high'],
                low=df_motif['low'],
                close=df_motif['close'])])

fig.show()

FROM KAGGLE DATASET

DAILY TF

In [ ]:
symbol_ = 'ACCELYA'
path2 = '/Users/yash/Desktop/Trading/nse_data/Datasets/SCRIP'
Q_df = read_file_from_symbol2(symbol_, basedir=path2)
Q_df['pivot'] = (Q_df['high']+Q_df['low']+Q_df['close'])/3
start_ = '20210611'
end_ = '20210630'
field = 'close'

Q_df = pd.DataFrame(filter_df_dates(Q_df, start_, end_)[field])

In [ ]:
files = glob(path2+'/*')
match_dict_df = pd.DataFrame(columns = ['Symbol', 'd', 'idx', 1, 2])
e = []

In [ ]:
for f in files:
    try:
        symbol_ = f.split('/')[-1].split('.')[0]
        print(symbol_)
        df = read_file_from_symbol2(symbol_, path2)
        df = n_month_gain(df, 1)
        df = n_month_gain(df, 3)
        df = n_month_gain(df, 6)
        df = adr(df)
        df['pivot'] = (df['high']+df['low']+df['close'])/3
        T_df = pd.DataFrame(df[field])
        m = Q_df[field].size
        # To set the exclusion zone to zero, set the denominator to np.`inf`
        stumpy.config.STUMPY_EXCL_ZONE_DENOM = np.inf
        # print(Q_df.shape, T_df.shape)
        top_N = stumpy.match(
            Q_df[field],
            T_df[field],
            # max_distance=np.inf, # set the threshold to infinity include all subsequences      # find the top 1 match,
            max_matches=5
        )
        # print(len(top_N))
        for n in range(len(top_N)):
            temp = top_N[n]
            match_distance = temp[0]
            match_idx = temp[1]
            list_ = [symbol_, match_distance, match_idx, df.iloc[match_idx], df.iloc[match_idx+m]]
            match_dict_df.loc[len(match_dict_df)] = list_

        stumpy.config.STUMPY_EXCL_ZONE_DENOM = 4 # Reset the denominator to its default value
        print('***')
    except:
        e.append(symbol_)
        print('Error in', symbol_)

In [ ]:
match_dict_df = match_dict_df.sort_values('d')
match_dict_df['start_date'] = match_dict_df[1].apply(lambda x: x.Date)
for j in ['1M_low_gain', '3M_low_gain', '6M_low_gain', 'ADR%', 'Date']:
    match_dict_df[j] = match_dict_df[2].apply(lambda x: x[j])